In [7]:
import os
x=os.walk(r"./data")
x=list(x)
entries={}
it = 0
for i in range(1,len(x)) :

    entries[i]=x[i][0].split("\\")[-1]
    
print(entries)


{1: 'Acacia', 2: 'Adenanthera microsperma', 3: 'Adenium species', 4: 'Anacardium occidentale', 5: 'Annona squamosa', 6: 'Artocarpus altilis', 7: 'Artocarpus heterophyllus', 8: 'Barringtonia acutangula', 9: 'Cananga odorata', 10: 'Carica papaya', 11: 'Casuarina equisetifolia', 12: 'Cedrus', 13: 'Chrysophyllum cainino', 14: 'Citrus aurantiifolia', 15: 'Citrus grandis', 16: 'Cocos nucifera', 17: 'Dalbergia oliveri', 18: 'Delonix regia', 19: 'Dipterocarpus alatus', 20: 'Erythrina fusca', 21: 'Eucalyptus', 22: 'Ficus microcarpa', 23: 'Ficus racemosa', 24: 'Gmelina arborea Roxb', 25: 'Hevea brasiliensis', 26: 'Hopea', 27: 'Khaya senegalensis', 28: 'Khaya senegalensis A.Juss', 29: 'Lagerstroemia speciosa', 30: 'Magnolia alba', 31: 'Mangifera', 32: 'Melaleuca', 33: 'Melia azedarach', 34: 'Musa', 35: 'Nephelium lappaceum', 36: 'Persea', 37: 'Polyalthia longifolia', 38: 'Prunnus', 39: 'Prunus salicina', 40: 'Psidium guajava', 41: 'Pterocarpus macrocarpus', 42: 'Senna siamea', 43: 'Spondias mombi

In [8]:
import tensorflow as tf

In [9]:
def makeDefaultHiddenLayers(inputs):
        cnn = tf.keras.layers.Conv2D(16, (3, 3), padding="same",activation="relu")(inputs)
        # cnn = tf.keras.layers.core.Activation("relu")(cnn)
        cnn = tf.keras.layers.BatchNormalization(axis=-1)(cnn)
        cnn = tf.keras.layers.MaxPooling2D(pool_size=(3, 3))(cnn)
        cnn = tf.keras.layers.Dropout(0.25)(cnn)
        cnn = tf.keras.layers.Conv2D(32, (3, 3), padding="same")(cnn)
        cnn = tf.keras.layers.Activation("relu")(cnn)
        cnn = tf.keras.layers.BatchNormalization(axis=-1)(cnn)
        cnn = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(cnn)
        cnn = tf.keras.layers.Dropout(0.25)(cnn)
        cnn = tf.keras.layers.Conv2D(32, (3, 3), padding="same")(cnn)
        cnn = tf.keras.layers.Activation("relu")(cnn)
        cnn = tf.keras.layers.BatchNormalization(axis=-1)(cnn)
        cnn = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(cnn)
        cnn = tf.keras.layers.Dropout(0.25)(cnn)
        return cnn
def makeModal(inputs,class_dict):
    cnn=makeDefaultHiddenLayers(inputs)
    cnn = tf.keras.layers.Flatten()(cnn)
    cnn = tf.keras.layers.Dense(128)(cnn)
    cnn = tf.keras.layers.Activation("relu")(cnn)
    cnn = tf.keras.layers.BatchNormalization()(cnn)
    cnn = tf.keras.layers.Dropout(0.5)(cnn)
    cnn = tf.keras.layers.Dense(class_dict)(cnn)
    cnn = tf.keras.layers.Activation("softmax", name="Bark_Output")(cnn)

    return cnn
def assemble(width,height,class_dict):
    inputs = tf.keras.layers.Input(shape=(height, width, 3))
    cnnOutputLayers=makeModal(inputs,class_dict)
    model=tf.keras.models.Model(inputs=inputs,outputs = [cnnOutputLayers],name="face_net")
    return model


In [10]:
X=assemble(303, 404,50)
print(X.output_shape)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(None, 50)
